In [5]:
import pandas as pd
import numpy as np
import acquire as a
import prepare as p


import matplotlib.pyplot as plt
import seaborn as sb

import nltk
from nltk.corpus import stopwords
import unicodedata
import re
import requests
from bs4 import BeautifulSoup as bs
import json
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')
# nltk.download('all')



### Acquire

In [6]:
df = a.get_dataFrame()

In [7]:
df = p.basic_clean_df(df)

In [8]:
df

,language,readme_contents,readme_length
0,java,development moved httpsgithubcomjasonellejason...,50
1,java,like project show support starring repository ...,6691
2,java,circularimageview custom view circular image a...,2405
3,java,android system ui training android uiforkstar...,589
4,java,tianti httpsyuedubaiducomebookaefafbdfbafetian...,2862
...,...,...,...
145,python,convolutional neural network sentence classifi...,882
146,python,alexapi use new alexapi httpsgithubcomalexapia...,2841
147,python,introduction hangupsbot chat bot designed work...,5440
148,python,tool allows scan multiple host heartbleed effi...,1065


In [4]:
df.language.value_counts()

Java          50
JavaScript    50
Python        50
Name: language, dtype: int64

In [5]:
df.isnull().sum()

repo               0
language           0
readme_contents    0
dtype: int64

### Prepare

In [6]:
from nltk.corpus import stopwords

In [7]:
basic_clean_df2(df)

,repo,language,readme_contents
0,jasonettejasonetteandroid,java,development moved httpsgithubcomjasonellejason...
1,vivcharrendererrecyclerviewadapter,java,like project show support starring repository ...
2,pkmmtecircularimageview,java,circularimageview custom view circular image a...
3,dclockandroidsystemuitraining,java,android system ui training android uiforkstar...
4,xujefftianti,java,tianti httpsyuedubaiducomebook7a5efa31fbd6195f...
...,...,...,...
145,alexanderrakhlincnnforsentenceclassificationin...,python,convolutional neural network sentence classifi...
146,alexapialexapideprecated,python,alexapi use new alexapi httpsgithubcomalexapia...
147,hangoutsbothangoutsbot,python,introduction hangupsbot chat bot designed work...
148,musalbasheartbleedmasstest,python,tool allows scan multiple host heartbleed effi...


In [8]:
# def readme_length(df, col):
#     """ This function takes in each README.md file and returns the word count for each file"""
#     length = []
#     for x in df[col]:
#         read_len= len(x)
#         length.append(read_len)
#     return length

In [9]:
df['readme_length']= readme_length(df,'readme_contents')

In [10]:
df

,repo,language,readme_contents,readme_length
0,Jasonette/JASONETTE-Android,Java,Development Moved to\nhttps://github.com/jason...,61
1,vivchar/RendererRecyclerViewAdapter,Java,If you like this project you can show support ...,8876
2,Pkmmte/CircularImageView,Java,CircularImageView\n=================\n\nCustom...,3599
3,D-clock/AndroidSystemUiTraining,Java,# Android System UI Training\n\n本代码库整理总结 Andro...,905
4,xujeff/tianti,Java,# 天梯（tianti）\n [天梯](https://yuedu.baidu.com/e...,4557
...,...,...,...,...
145,alexander-rakhlin/CNN-for-Sentence-Classificat...,Python,# Convolutional Neural Networks for Sentence C...,1200
146,alexa-pi/AlexaPiDEPRECATED,Python,# AlexaPi\n\n## Use the new AlexaPi https://gi...,4457
147,hangoutsbot/hangoutsbot,Python,# Introduction\n\nHangupsbot is a chat bot des...,7766
148,musalbas/heartbleed-masstest,Python,This tool allows you to scan multiple hosts fo...,1813


### Explore 

Will come back to this. Moving on to modeling

### Split

In [11]:
from sklearn.model_selection import train_test_split

In [10]:
train, val, test = p.train_val_test(df, stratify=True, target='language')

In [11]:
train.shape, val.shape, test.shape

((84, 3), (36, 3), (30, 3))

In [16]:
train.head()

,repo,language,readme_contents,readme_length
1,vivchar/RendererRecyclerViewAdapter,Java,If you like this project you can show support ...,8876
95,maxzhang/maxzhang.github.com,JavaScript,maxzhang.github.io\n==================\n\n> Th...,102
99,svrcekmichal/redux-axios-middleware,JavaScript,# redux-axios-middleware\n\n[![npm version](ht...,11542
49,liuyanggithub/SuperMvp,Java,# SuperMvp\n## **MVP + RxJava3 + Retrofit + G...,3438
104,some-programs/exitwp,Python,######\nExitwp\n######\n\nExitwp is tool for m...,3888


In [12]:
# X= df.readme_contents
# y= df.language

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=123)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size =0.2, random_state=123)



In [14]:
# X_train.shape

In [13]:
x_train = train['readme_contents']
y_train = train.language

x_val = val['readme_contents']
y_val = val.language


x_test = test['readme_contents']
y_test = test.language

### Modeling

**Establish Baseline**

In [16]:
baseline_acc = (train.language == 'python').mean()
baseline_acc

0.3333333333333333

In [17]:
tv = TfidfVectorizer()
cv= CountVectorizer()
dt = DecisionTreeClassifier(max_depth=5)
rf= RandomForestClassifier(max_depth= 5)
knn = KNeighborsClassifier(n_neighbors= 5)

**DecisionTree**

In [18]:
#Count Vectorizer
train_cv= cv.fit_transform(x_train)
dt.fit(train_cv, y_train)
print(f'Decision Tree Model score on Train set: {round(dt.score(train_cv, y_train),2)}')
val_cv= cv.transform(x_val)
print(f'Decision Tree Model score on Validate set {round(dt.score(val_cv, y_val),2)}')

Decision Tree Model score on Train set: 0.89
Decision Tree Model score on Validate set 0.61


In [19]:
#TfidVectorizer
train_tv= tv.fit_transform(x_train)
dt.fit(train_tv, y_train)
print(f'Decision Tree Model score on Train set: {round(dt.score(train_tv, y_train),3)}')
val_tv= tv.transform(x_val)
print(f'Decision Tree Model score on Validate set {round(dt.score(val_tv, y_val),3)}')

Decision Tree Model score on Train set: 0.905
Decision Tree Model score on Validate set 0.667


**RandomForest**

In [20]:
train_tv= tv.fit_transform(x_train)
rf.fit(train_tv, y_train)
print(f'RandomForest Model score on Train set: {round(rf.score(train_tv, y_train),3)}')
val_tv= tv.transform(x_val)
print(f'RandomForest Model score on Validate set {round(rf.score(val_tv, y_val),3)}')

RandomForest Model score on Train set: 0.976
RandomForest Model score on Validate set 0.75


**KNearest Neighbors**

In [54]:
train_tv= tv.fit_transform(x_train)
knn.fit(train_tv, y_train)
print(f'RandomForest Model score on Train set: {round(knn.score(train_tv, y_train),2)}')
val_tv= tv.transform(x_val)
print(f'RandomForest Model score on Validate set {round(knn.score(val_tv, y_val),2)}')

x_test_bow = tfidf.transform(x_test)
nb.score(x_test_bow, y_test)
print('Accuracy of Random Forest classifier on test set: {:.2f}'
     .format(knn.score(x_test_bow, y_test)))

RandomForest Model score on Train set: 0.81
RandomForest Model score on Validate set 0.81
Accuracy of Random Forest classifier on test set: 0.90


**Naive Bayes**

In [50]:
from sklearn.naive_bayes import MultinomialNB
tfidf = TfidfVectorizer()
nb = MultinomialNB()
X_bow = tfidf.fit_transform(x_train)
nb.fit(X_bow, y_train)
nb.score(X_bow, y_train)
x_val_bow = tfidf.transform(x_val)
nb.score(x_val_bow, y_val)
print('Accuracy of Naive_bayes classifier on training set: {:.2f}'
     .format(nb.score(X_bow, y_train)))
print('Accuracy of Naive_bayes classifier on validation set: {:.2f}'
     .format(nb.score(x_val_bow, y_val)))


x_test_bow = tfidf.transform(x_test)
nb.score(x_test_bow, y_test)
print('Accuracy of Naive_bayes classifier on test set: {:.2f}'
     .format(nb.score(x_test_bow, y_test)))


Accuracy of Naive_bayes classifier on training set: 1.00
Accuracy of Naive_bayes classifier on validation set: 0.89
Accuracy of Naive_bayes classifier on test set: 0.87


**Model Comparison**

In [48]:
import sklearn.metrics as m
def metrics_gala(train, val):
    results = {}
    
    tv = TfidfVectorizer()
    dt = DecisionTreeClassifier(max_depth=5)
    rf= RandomForestClassifier(max_depth= 5)
    knn = KNeighborsClassifier(n_neighbors= 5)

    baseline_acc = round((train.language == 'python').mean(),2)
    results['baseline'] = {'train_acc':baseline_acc}
    

    #TfidVectorizer on DT
    train_tv= tv.fit_transform(x_train)
    dt.fit(train_tv, y_train)
    val_tv= tv.transform(x_val)
    dt_train=round(dt.score(train_tv, y_train),2)
    dt_val=round(dt.score(val_tv, y_val),2)
    
    results['DecisionTree']={'train_acc': dt_train,
                            ' val_acc':dt_val,
                            'difference': dt_train-dt_val}
    
    #TfidVectorizer on RF
    train_tv= tv.fit_transform(x_train)
    rf.fit(train_tv, y_train)
    val_tv= tv.transform(x_val)
    rf_train=round(rf.score(train_tv, y_train),2)
    rf_val=round(rf.score(val_tv, y_val),2)
    results['RandomForest']={'train_acc': round(rf.score(train_tv, y_train),2),
                            ' val_acc':round(rf.score(val_tv, y_val),2),
                            'difference': rf_train-rf_val}
    
    #TfidVectorizer on KNN
    train_tv= tv.fit_transform(x_train)
    knn.fit(train_tv, y_train)
    val_tv= tv.transform(x_val)
    knn_train=round(knn.score(train_tv, y_train),2)
    knn_val=round(knn.score(val_tv, y_val),2)
    results['KNearestNeighbor']={'train_acc':knn_train,
                                ' val_acc':knn_val,
                                'difference': knn_train-knn_val}
    
    tfidf = TfidfVectorizer()
    nb = MultinomialNB()
    X_bow = tfidf.fit_transform(x_train)
    nb.fit(X_bow, y_train)
    nb_train = nb.score(X_bow, y_train)
    x_val_bow = tfidf.transform(x_val)
    nb_val = nb.score(x_val_bow, y_val)
    results['Naive Bayes']={'train_acc': round(nb_train,2),
                                ' val_acc':round(nb_val,2),
                                'difference': nb_train-nb_val}
    
    


    return pd.DataFrame(results).T

In [49]:
metrics_gala(train, val)

,train_acc,val_acc,difference
baseline,0.33,NaN,NaN
DecisionTree,0.90,0.67,0.230000
RandomForest,0.96,0.67,0.290000
KNearestNeighbor,0.81,0.81,0.000000
Naive Bayes,1.00,0.89,0.111111
